OData API

https://oda.ft.dk/Home/OdataQuery

https://www.odata.org/

https://www.ft.dk/-/media/sites/ft/pdf/dokumenter/aabne-data/oda-browser_brugervejledning.ashx

https://www.odata.org/documentation/odata-version-3-0/url-conventions/

https://www.odata.org/documentation/odata-version-3-0/odata-version-3-0-core-protocol/

Source root Link: "https://oda.ft.dk/api/"

ftp://oda.ft.dk/ODAXML/Referat/samling/


## Folketinget data

The data related to the meetings in Folketinget is collected following the guide on Folketinget's homepage:
https://www.ft.dk/-/media/sites/ft/pdf/dokumenter/aabne-data/oda-browser_brugervejledning.ashx.
Transcripts from all meetings going back to 2009 are available for download in xml format using a login described in the guide and some specific ftp software. Because it is interesting to compare topics from Twitter with topics in Folketinget, only the transcripts from the same period as the tweets are used for further analysis. 71% of the available tweets are posted in 2018 and after.

The years in Folketinget do not follow the calender year, as they start and end the first Tuesday of October. So, the meetings from 2017 are actually from October 2017 to October 2018. The data from 2017 are kept for the purpose of having slightly more data and because it is assumed that topics vary little enought for it to still be relevant.

In total 395 meeting stranscripts amounting to 218Mb of data has been downloaded and parsed for analysis.

It is not stated how the transcripts are made, however, they are proofread before publication. 


In [ ]:
#pip install bs4 lxml

In [80]:
import requests
import urllib.request
import os
import io
import json

import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import lxml

import xml.etree.ElementTree as et 
from xml.dom import minidom

import re

import dateutil
import datetime as dt
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *

#pip install afinn
from afinn import Afinn

In [11]:
path = os.getcwd()

In [12]:
path = path+'/samling/'

In [13]:
path

'/Users/frederikkromannhansen/Documents/GitHub/politiker/samling/'

In [8]:
ls samling/20181

20181_M10_helemoedet.xml       20181_M52_helemoedet.xml
20181_M11_helemoedet.xml       20181_M53_helemoedet.xml
20181_M12_helemoedet.xml       20181_M54_helemoedet.xml
20181_M13_helemoedet.xml       20181_M55_helemoedet.xml
20181_M14_helemoedet.xml       20181_M56_helemoedet.xml
20181_M15_helemoedet.xml       20181_M57_helemoedet.xml
20181_M16_helemoedet.xml       20181_M58_helemoedet.xml
20181_M17_helemoedet.xml       20181_M59_helemoedet.xml
20181_M18_helemoedet.xml       20181_M5_helemoedet.xml
20181_M19_helemoedet.xml       20181_M60_helemoedet.xml
20181_M1_helemoedet.xml        20181_M61_helemoedet.xml
20181_M1_helemoedet_clean.xml  20181_M62_helemoedet.xml
20181_M20_helemoedet.xml       20181_M63_helemoedet.xml
20181_M21_helemoedet.xml       20181_M64_helemoedet.xml
20181_M22_helemoedet.xml       20181_M65_helemoedet.xml
20181_M23_helemoedet.xml       20181_M66_helemoedet.xml
20181_M24_helemoedet.xml       20181_M67_helemoedet.xml

20181_M26_helemoedet.xml       20181_M69_helemoe

# Parsing XML

In [15]:
for subdir, dir, files in os.walk(path):
    for file in files:
        filepath = subdir+'/' + file
        if filepath.endswith(".xml"):
            xmlTree = et.parse(filepath)
            
elemList = []

for elem in xmlTree.iter():
    elemList.append(elem.tag)

# Just printing out the result
from collections import Counter
Counter(elemList)

Counter({'Dokument': 1,
         'MetaMeeting': 1,
         'ParliamentarySession': 1,
         'ParliamentaryGroup': 1,
         'MeetingNumber': 1,
         'DateOfSitting': 1,
         'Location': 1,
         'EdixiDocLocation': 1,
         'AudioFileFolder': 1,
         'TitelGruppe': 1,
         'Titel': 1,
         'Linea': 126,
         'Char': 138,
         'UnderTitel': 1,
         'DagsordenPlan': 1,
         'Rubrica': 10,
         'PunktTekst': 8,
         'Exitus': 72,
         'DagsordenPunkt': 5,
         'MetaFTAgendaItem': 5,
         'ItemNo': 5,
         'FTCase': 5,
         'FTCaseNumber': 5,
         'FTCaseType': 5,
         'FTCaseStage': 5,
         'ShortTitle': 5,
         'Aktivitet': 10,
         'Tale': 12,
         'Taler': 12,
         'MetaSpeakerMP': 12,
         'OratorFirstName': 12,
         'OratorLastName': 12,
         'GroupNameShort': 12,
         'OratorRole': 12,
         'TalerTitel': 12,
         'TaleSegment': 12,
         'MetaSpeechSegme

In [28]:
for speech in root.iter('Tale'):
    tags=[node.tag for node in speech.iter()]

In [29]:
tags

['Tale',
 'Taler',
 'MetaSpeakerMP',
 'OratorFirstName',
 'OratorLastName',
 'GroupNameShort',
 'OratorRole',
 'TalerTitel',
 'Linea',
 'Char',
 'Char',
 'TaleSegment',
 'MetaSpeechSegment',
 'LastModified',
 'EdixiStatus',
 'StartDateTime',
 'TekstGruppe',
 'Exitus',
 'Linea',
 'Char',
 'Linea',
 'Char',
 'Linea',
 'Char']

In [20]:
df.head()

,MeetingID,ItemNo,StartDateTime,OratorFirstName,OratorLastName,GroupNameShort,OratorRole,TekstGruppe
0,66,0,2019-02-27T13:00:08,Pia,Kjærsgaard,DF,formand,Mødet er åbnet.
1,66,0,2019-02-27T13:01:34,Pia,Kjærsgaard,DF,formand,Det første spørgsmål er til justitsministeren...
2,66,0,2019-02-27T13:01:43,Pia,Kjærsgaard,DF,formand,Værsgo for at oplæse spørgsmålet.
3,66,0,2019-02-27T13:01:45,Christian,Langballe,DF,medlem,Spørgsmålet lyder: Vil regeringen stå fast på...
4,66,0,2019-02-27T13:02:05,Pia,Kjærsgaard,DF,formand,"Værsgo, ministeren."


In [141]:
starttid_vec = []
sluttid_vec=[]
navn_vec = []
efternavn_vec = []
tekst_vec = []
parti_vec = []
rolle_vec = []
meetingID_vec = []


# Go through all the folders and files containing transcripts
for subdir, dir, files in os.walk(path):
    for file in files:
        filepath = subdir+'/' + file
        if (filepath.endswith(".xml")) and not (filepath.endswith("clean.xml")):
            tree = et.parse(filepath)
            root = tree.getroot() 

            for meeting in root.iter('MeetingNumber'):
                meetingID = meeting.text
            # Do this if the speaker has an associated party, i.e. not a minister or the chairman
            for speech in root.iter('Tale'):

                tags=[node.tag for node in speech.iter()]
                if('TekstGruppe' in tags):
                    if('GroupNameShort' in tags):
                        for node in speech.iter():
                            if(node.tag == 'StartDateTime'):
                                starttid_vec.append(node.text)
                            elif(node.tag == 'OratorFirstName'):
                                navn_vec.append(node.text)
                            elif(node.tag == 'OratorLastName'):
                                efternavn_vec.append(node.text)
                            elif(node.tag == 'OratorRole'):
                                rolle_vec.append(node.text)
                            elif(node.tag=='TekstGruppe'):
                                # This tag is preculiar, as there are multiple children ending with 'Char' as the youngest.
                                # Concate all text within 'TekstGruppe'.
                                tekstgruppe = ''
                                for k in node.iter():
                                    if k.tag=="Char":
                                        tekstgruppe+=' '+k.text
                                tekst_vec.append(tekstgruppe)
                            elif(node.tag=='GroupNameShort'):
                                parti_vec.append(node.text)
                                
                                meetingID_vec.append(meetingID)


                            # If there are more text than speaker names, remove the excess.
                            # We will loose a few pieces of text but only around 1%
                            if len(tekst_vec)>len(navn_vec):
                                tekst_vec.pop()
                            if len(starttid_vec)>len(navn_vec):
                                starttid_vec.pop()

                    # Do this if the speaker is either a minister or the chairman
                    else:
                        parti_vec.append('Ukendt')
                        for node in speech.iter():
                            if(node.tag == 'StartDateTime'):
                                starttid_vec.append(node.text)
                            elif(node.tag == 'OratorFirstName'):
                                navn_vec.append(node.text)
                            elif(node.tag == 'OratorLastName'):
                                efternavn_vec.append(node.text)
                            elif(node.tag == 'OratorRole'):
                                rolle_vec.append(node.text)
                            elif(node.tag=='TekstGruppe'):
                                # This tag is preculiar, as there are multiple children ending with 'Char' as the youngest.
                                # Concate all text within 'TekstGruppe'.
                                tekstgruppe = ''
                                for k in node.iter():
                                    if k.tag=="Char":
                                        tekstgruppe+=' '+k.text
                                tekst_vec.append(tekstgruppe)
                                
                                meetingID_vec.append(meetingID)

                            # If there are more text than speaker names, remove the excess.
                            # We will loose a few pieces of text but only around 1%
                            if len(tekst_vec)>len(navn_vec):
                                tekst_vec.pop()
                            if len(starttid_vec)>len(navn_vec):
                                starttid_vec.pop()
                    


# Add everuthing to a dictionary and transform it into a Pandas dataframe
dictionary= {'MeetingID':meetingID_vec, 'StartDateTime':starttid_vec,'OratorFirstName':navn_vec,'OratorLastName':efternavn_vec,
                 'GroupNameShort':parti_vec,'OratorRole':rolle_vec ,'TekstGruppe':tekst_vec} # 'EndDateTime':sluttid_vec,
df = pd.DataFrame(dictionary)

print('In total, there are {} lines of text in the meetings.'.format(len(df)))
#df.to_csv(path[0:26]+'csv')

In total, there are 151571 lines of text in the meetings.


In [142]:
len(meetingID_vec)

151571

In [143]:
### Cleaning the data ###

# The chariman (formand) does not add value to the context as he or she only moderates the debate
# by passing on the word from person to person.
# Also some meta data is contained regarding when the meeting ended (MødeSlut). This is also irrelevant.
df=df[df['OratorRole']!='formand']
df=df[df['OratorRole']!='MødeSlut']
df['OratorRole'].dropna(axis=0,inplace=True)

df=df[df['GroupNameShort']!='UFG']
df=df[df['GroupNameShort']!='Pause']

df=df[df['TekstGruppe']!=' (Talen er under udarbejdelse)']

df.reset_index(drop=True,inplace=True)



# Generating a column of the full name of the speakers
df['FullName'] = df['OratorFirstName']+' '+df['OratorLastName']
df.drop(['OratorFirstName','OratorLastName'],axis='columns', inplace=True)

# Making start time into datetime64 format
df['StartDateTime'] = df['StartDateTime'].apply(lambda x: dateutil.parser.parse(x))

df['MeetingID']=df['MeetingID'].apply(lambda x: int(x))
#df['ItemNo']=df['ItemNo'].apply(lambda x: int(x))

In [144]:
df

,MeetingID,StartDateTime,GroupNameShort,OratorRole,TekstGruppe,FullName
0,66,2019-02-27 13:01:45,DF,medlem,Spørgsmålet lyder: Vil regeringen stå fast på...,Christian Langballe
1,66,2019-02-27 13:02:06,None,minister,Tak for det. Og tak for spørgsmålet. Fremmedk...,Søren Pape Poulsen
2,66,2019-02-27 13:03:40,DF,medlem,Det står tilbage at få besvaret nogle ting i ...,Christian Langballe
3,66,2019-02-27 13:04:55,None,minister,"Tak. Jeg skal prøve, så godt jeg kan – det va...",Søren Pape Poulsen
4,66,2019-02-27 13:05:49,DF,medlem,"Altså, jeg synes jo, at de her burkabrude, so...",Christian Langballe
...,...,...,...,...,...,...
71513,23,2019-11-20 16:22:31,V,medlem,Det kunne jeg godt tænke mig at spørge lidt m...,Tommy Ahlers
71514,23,2019-11-20 16:23:03,None,minister,"Ordføreren har jo ret i, at undersøgelser vis...",Peter Hummelgaard Thomsen
71515,23,2019-11-20 16:24:00,V,medlem,Et enkelt spørgsmål her til sidst: Håber mini...,Tommy Ahlers
71516,23,2019-11-20 16:24:11,None,minister,"Hvis ikke regeringen håbede, at det ville bli...",Peter Hummelgaard Thomsen


In [ ]:
df_agg_party_date = df.groupby(['GroupNameShort', df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day])['TekstGruppe'].apply(lambda x : ' '.join(x))

In [ ]:
df_agg_party_date.sort_index().head()

In [ ]:
df_agg_name = df.groupby(['FullName'])['TekstGruppe'].apply(lambda x : ' '.join(x))
pd.DataFrame(df_agg_name).head()

In [ ]:
df_agg_date_party = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day, 'GroupNameShort'])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date_party).head()

In [123]:
df_agg_party = df.groupby(['GroupNameShort'])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_party).head()

,TekstGruppe
GroupNameShort,
ALT,"Det er helt afgørende, at vi forsøger at få e..."
DF,Spørgsmålet lyder: Vil regeringen stå fast på...
EL,"I går, den 18. marts, fik vi ministerens svar..."
IA,Mange tak. Inuit Ataqatigiit støtter også lov...
JF,Tak for det. Så går vi nogle luftlag ned til ...


In [ ]:
df['MeetingID']=df['MeetingID'].apply(lambda x: int(x))

In [ ]:
df.info()

In [43]:
df_agg_date = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date).head()

TekstGruppe
StartDateTime StartDateTime StartDateTime                                                   
2018          10            2               Deres Majestæt, Deres Kongelige Højheder. I e...
                            4               Tid er en underlig størrelse. Jeg er f.eks. s...
                            9               Tak for det, formand. Nu får jeg anledning ti...
                            10              Tak for det. Lovforslaget omfatter tre område...
                            11              Tak, hr. formand. Det her er to forslag, der ...

In [ ]:
df_agg_date_name = df.groupby([df['StartDateTime'].dt.year, df['StartDateTime'].dt.month, df['StartDateTime'].dt.day, 'FullName'])['TekstGruppe'].apply(lambda x : ' '.join(x)).sort_index()
pd.DataFrame(df_agg_date_name).head()

## Topics

In [145]:
stopwords = open('stopord.txt').read().splitlines() #Link: https://gist.github.com/berteltorp/0cf8a0c7afea7f25ed754f24cfc2467b

In [146]:
# convert the text to a tf-idf weighted term-document matrix
from __future__ import division

# Hyperparameters
corpus = 700000 #Der er ca. 66247 forskellige ord, efter stopord
l1_ratio = 0
max_iter = 50000
no_components = 20 #number of topics

# The words should appear at least 'min_df' times and at most 'max_df' pr document. Decimals indicate proportions
min_df = 2
max_df = 0.6

data=df_agg_date

vectorizer = TfidfVectorizer(max_features=corpus, min_df=min_df, max_df=max_df, stop_words=stopwords)

X = vectorizer.fit_transform(data)
 
idx_to_word = np.array(vectorizer.get_feature_names())


# apply Non-negative Matrix Factorization to get topics using TF-IDF
nmf = NMF(n_components=no_components, solver="mu", l1_ratio = l1_ratio, max_iter = max_iter, random_state=1)

W = nmf.fit_transform(X)

H = nmf.components_
 
# print the topics
 
for i, topic in enumerate(H):
 
    print("Topic {}: {}".format(i + 1, ", ".join([str(x) for x in idx_to_word[topic.argsort()[-10:]]])))

Topic 1: indvandrere, beslutningsforslaget, elever, arbejdskraft, asylansøgere, kriminalitet, politiet, udlændinge, grænsekontrol, flygtninge
Topic 2: finanslovsforslag, demografiske, velfærd, finansministeren, omstilling, færøerne, finansloven, ældreområdet, finanslov, grønland
Topic 3: fremmedkrigere, indfødsretsudvalget, udlændinge, statsborgere, kriminalitet, statsborgerskaber, indfødsret, statsborgerskabet, statsborger, statsborgerskab
Topic 4: lønmodtagere, smitte, krisen, hjælpepakker, krise, corona, 19, su, coronakrisen, covid
Topic 5: kina, arktisk, nordiske, nato, rusland, usa, arktiske, rigsfællesskabet, arktis, grønland
Topic 6: resolution, rohde, tyrkiets, messerschmidt, udenrigsministeren, parlamentet, syrien, tyrkiske, erdogan, tyrkiet
Topic 7: akutberedskabet, patienterne, sygehus, sundhedsreformen, sundhedsreform, læge, patienter, regionerne, læger, sundhedsvæsen
Topic 8: minkene, minkproduktion, ordre, dyr, nordjylland, dyrevelfærd, lovhjemmel, folkesundheden, minkavl

In [147]:
topics = []
for i in range(no_components):
    topics.append('Topic '+str(i+1))
    
topics

['Topic 1',
 'Topic 2',
 'Topic 3',
 'Topic 4',
 'Topic 5',
 'Topic 6',
 'Topic 7',
 'Topic 8',
 'Topic 9',
 'Topic 10',
 'Topic 11',
 'Topic 12',
 'Topic 13',
 'Topic 14',
 'Topic 15',
 'Topic 16',
 'Topic 17',
 'Topic 18',
 'Topic 19',
 'Topic 20']

In [55]:
df.head()

,MeetingID,StartDateTime,OratorFirstName,OratorLastName,GroupNameShort,OratorRole,TekstGruppe,FullName
0,66,2019-02-27 13:01:45,Christian,Langballe,DF,medlem,Spørgsmålet lyder: Vil regeringen stå fast på...,Christian Langballe
1,66,2019-02-27 13:02:06,Søren Pape,Poulsen,None,minister,Tak for det. Og tak for spørgsmålet. Fremmedk...,Søren Pape Poulsen
2,66,2019-02-27 13:03:40,Christian,Langballe,DF,medlem,Det står tilbage at få besvaret nogle ting i ...,Christian Langballe
3,66,2019-02-27 13:04:55,Søren Pape,Poulsen,None,minister,"Tak. Jeg skal prøve, så godt jeg kan – det va...",Søren Pape Poulsen
4,66,2019-02-27 13:05:49,Christian,Langballe,DF,medlem,"Altså, jeg synes jo, at de her burkabrude, so...",Christian Langballe


In [148]:
# List of words
idx_to_word

array(['00', '004', '01', ..., 'øvet', 'øvre', 'øvrig'], dtype='<U34')

In [149]:
# Last topic's most defining words
print(topic.argsort()[-10:])
print(idx_to_word[topic.argsort()[-10:]])

[44400  1609 17778 15673 13592  3786 25001 42846 40880 28192]
['valgforbund' 'aldersformand' 'højheder' 'gruppernes' 'fremgang'
 'begærer' 'majestæt' 'udpeget' 'tingsekretærer' 'næstformand']


In [150]:
print('There are ', len(data), ' rows in this aggregation')
print('and from these, ', W.shape[1], ' topics are computed.')
print()
print('There are ', len(idx_to_word),' words in the corpus.')
print(H.shape)

There are  243  rows in this aggregation
and from these,  20  topics are computed.

There are  46820  words in the corpus.
(20, 46820)


In [152]:
pd.concat([data, pd.DataFrame(W)], axis=0, join='outer', ignore_index=False, keys=None,
          levels=None, names=None, verify_integrity=False, copy=True).reset_index()

,index,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,"(2018, 10, 2)","Deres Majestæt, Deres Kongelige Højheder. I e...",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,"(2018, 10, 4)",Tid er en underlig størrelse. Jeg er f.eks. s...,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,"(2018, 10, 9)","Tak for det, formand. Nu får jeg anledning ti...",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,"(2018, 10, 10)",Tak for det. Lovforslaget omfatter tre område...,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,"(2018, 10, 11)","Tak, hr. formand. Det her er to forslag, der ...",nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,238,0.00397032,0.00000000,0.00000000,0.05599916,0.00000000,0.00182838,0.00000000,0.51323453,0.00000000,...,0.00709169,0.00000000,0.00000000,0.00461839,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
482,239,0.06505864,0.00000000,0.00000000,0.00000000,0.00420468,0.00000095,0.00000000,0.10972908,0.00000000,...,0.00100663,0.01400903,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.05996560,0.00373571,0.01488444
483,240,0.00000001,0.14391073,0.00000000,0.03029630,0.00000000,0.00000000,0.02474344,0.00000000,0.00000000,...,0.16614479,0.00000000,0.00000423,0.00507224,0.00000226,0.00000000,0.00000000,0.18929768,0.00000000,0.00289284
484,241,0.00000001,0.00000114,0.00000000,0.00000000,0.00000006,0.00000000,0.00000000,0.00000000,0.00000000,...,0.01274073,0.00000000,0.00001936,0.79871243,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000




W = Topics og tilhørende vægte. Den her linje har det her topic
H = Ord og hvor meget de hver passer i hvert topic. Det her ord er så meget topic 1 og så meget topic 2

[Navn, Tekst, 

## Sentiment

In [81]:
afinn = Afinn(language='da', emoticons=True)

In [ ]:
afinn.score('Hvis ikke det er det mest afskyelige flueknepperi...')

In [90]:
df['Sentiment'] = df['TekstGruppe'].apply(lambda x: afinn.score(x))

In [91]:
df

,MeetingID,StartDateTime,GroupNameShort,OratorRole,TekstGruppe,FullName,Sentiment
0,66,2019-02-27 13:01:45,DF,medlem,Spørgsmålet lyder: Vil regeringen stå fast på...,Christian Langballe,-1.00000000
1,66,2019-02-27 13:02:06,None,minister,Tak for det. Og tak for spørgsmålet. Fremmedk...,Søren Pape Poulsen,13.00000000
2,66,2019-02-27 13:03:40,DF,medlem,Det står tilbage at få besvaret nogle ting i ...,Christian Langballe,1.00000000
3,66,2019-02-27 13:04:55,None,minister,"Tak. Jeg skal prøve, så godt jeg kan – det va...",Søren Pape Poulsen,4.00000000
4,66,2019-02-27 13:05:49,DF,medlem,"Altså, jeg synes jo, at de her burkabrude, so...",Christian Langballe,-2.00000000
...,...,...,...,...,...,...,...
52773,23,2019-11-20 16:22:31,V,medlem,Det kunne jeg godt tænke mig at spørge lidt m...,Tommy Ahlers,-6.00000000
52774,23,2019-11-20 16:23:03,None,minister,"Ordføreren har jo ret i, at undersøgelser vis...",Peter Hummelgaard Thomsen,2.00000000
52775,23,2019-11-20 16:24:00,V,medlem,Et enkelt spørgsmål her til sidst: Håber mini...,Tommy Ahlers,2.00000000
52776,23,2019-11-20 16:24:11,None,minister,"Hvis ikke regeringen håbede, at det ville bli...",Peter Hummelgaard Thomsen,4.00000000


In [97]:
afinn.score('afskyelige afskyelige afskyelige afskyelige afskyelige afskyelige afskyelige afskyelige')

-24.0

In [98]:
pd.DataFrame(H[19][topic.argsort()[-10:]])

,0
0,0.15396326
1,0.16099227
2,0.16099227
3,0.16099227
4,0.16529477
5,0.18657274
6,0.20013040
7,0.28731314
8,0.32198454
9,0.33248969


In [67]:
pd.set_option('display.float_format', lambda x: '%.8f' % x)
#pd.reset_option('display.float_format')
pd.DataFrame(W)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.00000000,0.00918551,0.00000000,0.00000000,0.03081518,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00046138,0.00000000,0.00000000,0.00000000,0.00000000,0.00043800,0.00000000,0.00000000,0.00045685,0.46326379
1,0.09321672,0.26203324,0.02758924,0.01297696,0.00000000,0.00000000,0.05678649,0.00000000,0.00000000,0.00000000,0.07023165,0.00509071,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.01815251
2,0.00000050,0.00403315,0.00188360,0.00728490,0.38882191,0.00000000,0.04968896,0.00051120,0.03830467,0.09392750,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000044,0.00000000,0.00000000,0.00000000
3,0.04685032,0.00000000,0.00000000,0.00000000,0.03224072,0.03612210,0.03746842,0.03170184,0.00887303,0.00000000,0.00000000,0.08732873,0.00000711,0.00163095,0.00000000,0.00000000,0.00000000,0.00000077,0.06249471,0.00000000
4,0.04859421,0.00080160,0.00000000,0.00000000,0.01104030,0.00000000,0.00000000,0.00000000,0.33839674,0.05016939,0.00000000,0.00000000,0.00000000,0.00000004,0.01209763,0.00000000,0.00017478,0.00000000,0.00000000,0.00271968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,0.00397032,0.00000000,0.00000000,0.05599916,0.00000000,0.00182838,0.00000000,0.51323453,0.00000000,0.00786037,0.00709169,0.00000000,0.00000000,0.00461839,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000
239,0.06505864,0.00000000,0.00000000,0.00000000,0.00420468,0.00000095,0.00000000,0.10972908,0.00000000,0.00404133,0.00100663,0.01400903,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.05996560,0.00373571,0.01488444
240,0.00000001,0.14391073,0.00000000,0.03029630,0.00000000,0.00000000,0.02474344,0.00000000,0.00000000,0.00000000,0.16614479,0.00000000,0.00000423,0.00507224,0.00000226,0.00000000,0.00000000,0.18929768,0.00000000,0.00289284
241,0.00000001,0.00000114,0.00000000,0.00000000,0.00000006,0.00000000,0.00000000,0.00000000,0.00000000,0.01416524,0.01274073,0.00000000,0.00001936,0.79871243,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000


In [ ]:
pd.DataFrame(H)

In [ ]:
# Write to a .csv file

#with open('topics_and_weights.csv', 'w', newline='', encoding='utf-16') as csvfile:
 #   topicwriter = csv.writer(csvfile, delimiter=',',
  #                          quotechar='"', quoting=csv.QUOTE_MINIMAL)
# Create the header
   # topicwriter.writerow(['Topic','Word','Weight'])

In [ ]:
# print the topics for tableau
 
#for i, topic in enumerate(H):
 
 #   max_weights = topic.argsort()[-10:]
  #  words = [str(x) for x in idx_to_word[topic.argsort()[-10:]]]
     
   # for count, item in enumerate(max_weights):
        
    #    with open('topics_and_weights.csv', 'a', newline='', encoding='utf-16') as csvfile:
     #       topicwriter = csv.writer(csvfile, delimiter=',',
      #                      quotechar='"', quoting=csv.QUOTE_MINIMAL)
       #     topicwriter.writerow(["Topic"+str(i+1) ,words[count],H[i,item]])

In [ ]:
#stopwords_edit = open('stopord.txt','a')
#stopwords_edit.write('\nenhedslisten')
#stopwords_edit.close()

#### Meetings text statistics (FREDERIK)

In [125]:
df['Sentiment'] = df['TekstGruppe'].apply(lambda x: afinn.score(x))

In [132]:
df.groupby('GroupNameShort').agg({"TekstGruppe": (lambda column: "".join(column)), 'Sentiment': (lambda column: sum(column))})

,TekstGruppe,Sentiment
GroupNameShort,,
ALT,"Det er helt afgørende, at vi forsøger at få e...",15237.00000000
DF,Spørgsmålet lyder: Vil regeringen stå fast på...,41180.00000000
EL,"I går, den 18. marts, fik vi ministerens svar...",27094.00000000
IA,Mange tak. Inuit Ataqatigiit støtter også lov...,2692.00000000
JF,Tak for det. Så går vi nogle luftlag ned til ...,620.00000000
KF,"Tak for det. Lovforslaget her handler jo, som...",14671.00000000
LA,"Tak for det. Der er formentlig ikke nogen, de...",13322.00000000
NB,Jeg skal også gøre det ganske kort. Nye Borge...,9056.00000000
NQ,"Grønland er i forandring, rigsfællesskabet er...",141.00000000


In [134]:
all_text = df[['GroupNameShort','TekstGruppe']]

In [135]:
all_text

,GroupNameShort,TekstGruppe
0,DF,Spørgsmålet lyder: Vil regeringen stå fast på...
1,None,Tak for det. Og tak for spørgsmålet. Fremmedk...
2,DF,Det står tilbage at få besvaret nogle ting i ...
3,None,"Tak. Jeg skal prøve, så godt jeg kan – det va..."
4,DF,"Altså, jeg synes jo, at de her burkabrude, so..."
...,...,...
52773,V,Det kunne jeg godt tænke mig at spørge lidt m...
52774,None,"Ordføreren har jo ret i, at undersøgelser vis..."
52775,V,Et enkelt spørgsmål her til sidst: Håber mini...
52776,None,"Hvis ikke regeringen håbede, at det ville bli..."


In [137]:
for party in set(df['GroupNameShort']):
    for i in df[df['GroupNameShort']==party]['TekstGruppe']:
        print(i)

 Jeg skal også gøre det ganske kort. Nye Borgerlige kan støtte dette yderst interessante og spændende lovforslag. Vi kan selvfølgelig støtte forslaget. Tak.
 Tak. Tak for ordet. Og tak for redegørelsen. Det var interessant at læse gennemgangen af aktiviteter igennem, som foregår under Nordisk Ministerråds budget. Nordisk samarbejde er en god ting, som vi skal fastholde. Der er mange af aktiviteterne, der er fornuftige og giver mening. Hvor vi med fordel kan samarbejde med de nordiske lande, skal vi naturligvis gøre det. Af de 12 overordnede mål, som indgår i visionen for det nordiske samarbejde frem mod 2030, indgår klima og/eller miljø i de 8. Det er bestemt også vigtigt, med der er jo også mange andre emner, man kan beskæftige sig med. Jeg kan nævne som eksempler på nogle fornuftige tiltag i redegørelsen – det er ikke udtømmende, men bare nogle eksempler – at der har været erfaringsudveksling under covid-19-krisen, samarbejde om grænseoverskridende kriminalitet og terrorbekæmpelse, o

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 Tak. Jeg vil stille hr. Sikandar Siddique et spørgsmål – jeg tror faktisk, jeg siger det rigtigt, jeg reddede mig. Det er ikke et polemisk spørgsmål, men jeg vil bare gerne lige spørge: Mener Alternativet, at hvis flere i ghettoerne eller i parallelsamfundene får dansk statsborgerskab, gør man simpelt hen op med det? Var det sådan, jeg skulle forstå det? Er det bare et spørgsmål om, at folk, der bor i Vollsmose, bare får dansk statsborgerskab, og så slipper man sådan set for de problemer, der er, i nogle områder?
 Medgiver ordføreren så, at der gennem tiden har været folk, der har fået dansk statsborgerskab, som ikke er særlig integreret, eller – hvad kan man sige – som lever lidt deres eget liv, selv om de har dansk statsborgerskab, og at det ikke alene er det, der har løst det?
 Godt, jamen så forstår jeg på formanden, at man hellere må være lidt hurtig. Først vil jeg selvfølgelig sige tak til Venstre for også at være med i et bredt forslag til vedtagelse på det her område. Det er v

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
Dict = dict()

for party in set(df['GroupNameShort']):
    for i in df[df['GroupNameShort']==party]['TekstGruppe']):
        if party in Dict:
            # append the new number to the existing array at this slot
            Dict[party].extend(i)
        else:
            # create a new array in this slot
            Dict[party[0]] = [party[1]]

In [ ]:
scores = []

for party in set(df['GroupNameShort']):
    if df[df['GroupNameShort']==party]:
        scores.append(df.apply(lambda x: affin(score(x))))
        box_labels.append(party)

In [ ]:
fig = plt.subplots(figsize=(15,10))
plt.tick_params(labelsize=16)
plt.xticks(rotation=90)
plt.ylabel('Sentiment score')
plt.title("Boxplot for party sentiment")
plt.boxplot(_scores, labels=box_labels)
plt.show()

In [ ]:
#### Plotting a relationship graph# for each party get statistics and make plot
box_labels = []
_scores = []

for key, value in total_sentiment.items():
    box_labels.append(key)
    score_l = []
    for inner in value.values():
        score = inner['scores']
        #print(score)
        score_l.extend(score)
    _scores.append(score_l)
print(len(_scores))
#The results are a bit mixed, we see dansk folkeparti agreeing the most with det konservative, and det konservative to venstre as these are all blue block members, however enhedslisten agrees most with Radikale, which is not the same block. Same thing can be said for Venstre -> Radikale and Liberal -> Radikale. This could suggest that the current block politic strategy might be a bit flawed. However it must also be considered that there are a lot of unknown parties so the truth could be burried in decrypting this data.

print((_scores))

fig = plt.subplots(figsize=(15,10))
plt.tick_params(labelsize=16)
plt.xticks(rotation=90)
plt.ylabel('Sentiment score')
plt.title("Boxplot for party tweet sentiment")
plt.boxplot(_scores, labels=box_labels)
plt.show()